# Image Explainability with Captum - Tutorial

This notebook demonstrates how to use the modular `image.py` functions for explainability analysis.

## Method 1: Using the High-Level Function

The easiest way - analyze all images with all methods:

In [ ]:
# Example 1: Using the High-Level Function
import sys
import os
sys.path.insert(0, './src')

from images import explain_model_predictions

print("=" * 80)
print("EXAMPLE 1: High-Level Function Usage")
print("=" * 80)

# Check if images exist in common directories
possible_paths = ['./data/fracture_samples', './data/generic_samples', './data/sample_images', './images/', './samples/']
data_path = None

for path in possible_paths:
    if os.path.exists(path) and os.path.isdir(path):
        # Check if directory has images
        image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
        images = [f for f in os.listdir(path) if f.lower().endswith(image_extensions)]
        if images:
            data_path = path
            print(f"\n✓ Found images in: {path}")
            break

if data_path:
    print(f"\nAnalyzing images in {data_path} with all methods...")
    print("(Processing up to 2 images for demo)\n")
    
    # Run the complete analysis
    explain_model_predictions(
        model_path='microsoft/resnet-50',
        data_path=data_path,
        output_dir='./outputs/images',
        num_samples=2,               # Limit to 2 images for demo
        methods=None                 # Use ALL methods
    )
    
    print("\n✓ Analysis complete! Check ./outputs/images/ for results")
else:
    print("\nℹ No images found in ", possible_paths)
    print("\n USAGE EXAMPLE (add images to run):")
    print("""
explain_model_predictions(
    model_path='microsoft/resnet-50',
    data_path='./my_images',        # Directory with images
    output_dir='./outputs/images',
    num_samples=None,               # Process ALL images (default)
    methods=None                    # Use ALL methods (default)
)
    """)
    print("✓ This will analyze all images with all three explainability methods")
    print("✓ Works with both directories and single image files")

EXAMPLE 1: High-Level Function Usage

✓ Found images in: ./data/fracture_samples

Analyzing images in ./data/fracture_samples with all methods...
(Processing up to 2 images for demo)

Selected methods: gradientshap, integratedgradients, saliency
Using device: cpu
Loading model: microsoft/resnet-50
Directory detected: ./data/fracture_samples

Found 2 images to analyze

EXPLAINABILITY ANALYSIS RESULTS

--- Image 1/2: brick_texture.png ---
Directory detected: ./data/fracture_samples

Found 2 images to analyze

EXPLAINABILITY ANALYSIS RESULTS

--- Image 1/2: brick_texture.png ---
Predicted class: 828 (confidence: 0.4866)
Computing GradientShap attributions...
Predicted class: 828 (confidence: 0.4866)
Computing GradientShap attributions...
GradientShap attribution range: [0.0001, 0.0962]
GradientShap mean absolute attribution: 0.0059
Computing Integrated Gradients attributions...
GradientShap attribution range: [0.0001, 0.0962]
GradientShap mean absolute attribution: 0.0059
Computing Integr

## Method 2: Using Specific Explainability Methods

Choose which methods to apply:

In [ ]:
# Example 2: Analyze with Specific Methods
import sys
import os
sys.path.insert(0, './src')

from images import explain_model_predictions

print("=" * 80)
print("EXAMPLE 2: Selective Method Usage")
print("=" * 80)

# Check for a single image file
possible_files = []
for ext in ['.jpg', '.jpeg', '.png']:
    for path in ['./images/', './data/', './samples/', './']:
        if os.path.exists(path):
            files = [os.path.join(path, f) for f in os.listdir(path) 
                    if f.lower().endswith(ext) and os.path.isfile(os.path.join(path, f))]
            if files:
                possible_files.extend(files[:1])  # Take first match
                break
    if possible_files:
        break

if possible_files:
    img_path = possible_files[0]
    print(f"\n✓ Found image: {img_path}")
    print(f"\nAnalyzing with specific methods: GradientShap and Saliency")
    print("(Skipping Integrated Gradients for faster processing)\n")
    
    # Run with selected methods
    explain_model_predictions(
        model_path='microsoft/resnet-50',
        data_path=img_path,                      # Single image file
        output_dir='./method_comparison',
        methods=['gradientshap', 'saliency']     # Only these methods
    )
    
    print("\n✓ Analysis complete! Check ./method_comparison/ for results")
else:
    print("\nℹ No images found. Add an image to run this example.")
    print("\n USAGE EXAMPLE:")
    print("""
explain_model_predictions(
    model_path='microsoft/resnet-50',
    data_path='./photo.jpg',                     # Single image file
    output_dir='./outputs',
    methods=['gradientshap', 'saliency']         # Only these methods
)
    """)
    print("✓ Works with single image files")
    print("✓ Choose which explainability methods to apply")
    print("✓ Available methods: 'gradientshap', 'integratedgradients', 'saliency'")

EXAMPLE 2: Selective Method Usage

ℹ No images found. Add an image to run this example.

📝 USAGE EXAMPLE:

explain_model_predictions(
    model_path='microsoft/resnet-50',
    data_path='./photo.jpg',                     # Single image file
    output_dir='./outputs',
    methods=['gradientshap', 'saliency']         # Only these methods
)
    
✓ Works with single image files
✓ Choose which explainability methods to apply
✓ Available methods: 'gradientshap', 'integratedgradients', 'saliency'


## Method 3: Using Individual Functions (Advanced)

Build custom workflows with modular functions:

In [20]:
# Example 3: Advanced - Build custom workflows with modular functions
import sys
import os
sys.path.insert(0, './src')

from helpers.image_utils import (
    load_model,
    get_image_files,
    create_transform,
    load_and_preprocess_image,
    get_model_prediction
)
from images import (
    compute_gradient_shap,
    compute_saliency,
    save_attribution_visualization
)

print("=" * 80)
print("EXAMPLE 3: Custom Modular Workflow")
print("=" * 80)

# Step 1: Load model once (reuse for multiple images)
print("\n[Step 1] Loading model...")
model, processor, device = load_model('microsoft/resnet-50')
print(f"✓ Model loaded on {device}")

# Step 2: Get images (you can change this path to your images)
print("\n[Step 2] Getting image files...")
# For demo, we'll check if there are any images in common directories
possible_paths = ['./images/', './data/', './samples/']
image_files = []
for path in possible_paths:
    if os.path.exists(path):
        image_files = get_image_files(path, num_samples=3)
        if image_files:
            print(f"✓ Found {len(image_files)} images in {path}")
            break

if not image_files:
    print("ℹ No images found in common directories (./images/, ./data/, ./samples/)")
    print("  To run this example, add images to one of these directories")
    print("  For now, showing the workflow structure:")
    print("\n  image_files = get_image_files('./images/', num_samples=10)")
    print("  transform = create_transform(processor)")
else:
    # Step 3: Create transform once
    print("\n[Step 3] Creating preprocessing transform...")
    transform = create_transform(processor)
    print("✓ Transform created")

    # Step 4: Process each image with custom logic
    print("\n[Step 4] Processing images with custom logic...")
    print("  Strategy: Use expensive GradientShap only for high-confidence predictions")
    print("            Use faster Saliency for low-confidence predictions\n")
    
    os.makedirs('./custom_outputs', exist_ok=True)
    
    for idx, img_path in enumerate(image_files):
        img_name = os.path.basename(img_path)
        print(f"\n  Image {idx+1}/{len(image_files)}: {img_name}")
        
        # Load and preprocess
        original_img, input_tensor = load_and_preprocess_image(img_path, transform, device)
        
        # Get prediction
        pred_class, pred_prob = get_model_prediction(model, input_tensor)
        print(f"    Prediction: Class {pred_class} (confidence: {pred_prob:.2%})")
        
        # Custom logic: Choose method based on confidence
        if pred_prob > 0.8:
            print(f"    High confidence → Using GradientShap (more accurate but slower)")
            result = compute_gradient_shap(model, input_tensor, pred_class, device, n_samples=30)
            method_name = "GradientShap"
        else:
            print(f"    Lower confidence → Using Saliency (faster)")
            result = compute_saliency(model, input_tensor, pred_class, device)
            method_name = "Saliency"
        
        # Save result
        if result:
            output_path = f'./custom_outputs/{os.path.splitext(img_name)[0]}_{method_name.lower()}.png'
            save_attribution_visualization(
                original_img, 
                result['attributions'], 
                method_name, 
                output_path, 
                pred_class, 
                pred_prob
            )
    
    print("\n" + "=" * 80)
    print("✓ Custom workflow completed!")
    print(f"✓ Results saved to: ./custom_outputs/")
    print("=" * 80)

EXAMPLE 3: Custom Modular Workflow

[Step 1] Loading model...
Using device: cpu
Loading model: microsoft/resnet-50
✓ Model loaded on cpu

[Step 2] Getting image files...
Directory detected: ./data/
ℹ No images found in common directories (./images/, ./data/, ./samples/)
  To run this example, add images to one of these directories
  For now, showing the workflow structure:

  image_files = get_image_files('./images/', num_samples=10)
  transform = create_transform(processor)
✓ Model loaded on cpu

[Step 2] Getting image files...
Directory detected: ./data/
ℹ No images found in common directories (./images/, ./data/, ./samples/)
  To run this example, add images to one of these directories
  For now, showing the workflow structure:

  image_files = get_image_files('./images/', num_samples=10)
  transform = create_transform(processor)


## Method 4: Demonstrate compute_integrated_gradients()

Use Integrated Gradients method directly:

In [21]:
# CLI Usage Examples (For Reference - Run these in terminal)
print("=" * 80)
print("COMMAND LINE INTERFACE (CLI) USAGE")
print("=" * 80)
print("\nNote: These commands are run in the terminal, not in this notebook\n")

examples = """
# Example 1: Analyze all images in directory with all methods
python src/images.py -data ./my_images/

# Example 2: Single image with specific method
python src/images.py -data ./photo.jpg -methods gradientshap

# Example 3: Limit number of images, specific methods
python src/images.py -data ./dataset/ -num_samples 5 -methods saliency integratedgradients

# Example 4: Different model
python src/images.py -data ./images/ -model google/vit-base-patch16-224

# Example 5: Custom output directory
python src/images.py -data ./images/ -outdir ./my_results/
"""

print(examples)
print("\n✓ Flexible command-line interface")
print("✓ Works with single files or directories")
print("✓ Choose methods and limit samples as needed")
print("\nℹ Note: CLI functionality requires removing __main__ block")
print("  (Already done - images.py is now a pure module)")

COMMAND LINE INTERFACE (CLI) USAGE

Note: These commands are run in the terminal, not in this notebook


# Example 1: Analyze all images in directory with all methods
python src/images.py -data ./my_images/

# Example 2: Single image with specific method
python src/images.py -data ./photo.jpg -methods gradientshap

# Example 3: Limit number of images, specific methods
python src/images.py -data ./dataset/ -num_samples 5 -methods saliency integratedgradients

# Example 4: Different model
python src/images.py -data ./images/ -model google/vit-base-patch16-224

# Example 5: Custom output directory
python src/images.py -data ./images/ -outdir ./my_results/


✓ Flexible command-line interface
✓ Works with single files or directories
✓ Choose methods and limit samples as needed

ℹ Note: CLI functionality requires removing __main__ block
  (Already done - images.py is now a pure module)


In [31]:
# Key Features Summary
print("=" * 80)
print("KEY FEATURES OF THE MODULAR DESIGN")
print("=" * 80)

features = """
MODULARITY
   - Each function has a single responsibility
   - Functions can be used independently or combined
   - Easy to test and maintain

FLEXIBILITY
   - Single image OR directory of images
   - Choose specific explainability methods
   - Custom workflows with individual functions
   - Limit number of images processed

REUSABILITY
   - Load model once, reuse for multiple images
   - Create preprocessing transform once
   - Mix and match functions as needed

EASE OF USE
   - High-level function for quick analysis
   - CLI for command-line usage  
   - Python API for programmatic access
   - Clear function signatures and returns

EXPLAINABILITY METHODS
   - GradientShap: Gradient-based Shapley values
   - Integrated Gradients: Path integration method
   - Saliency: Gradient magnitude highlighting
"""

print(features)

KEY FEATURES OF THE MODULAR DESIGN

MODULARITY
   - Each function has a single responsibility
   - Functions can be used independently or combined
   - Easy to test and maintain

FLEXIBILITY
   - Single image OR directory of images
   - Choose specific explainability methods
   - Custom workflows with individual functions
   - Limit number of images processed

REUSABILITY
   - Load model once, reuse for multiple images
   - Create preprocessing transform once
   - Mix and match functions as needed

EASE OF USE
   - High-level function for quick analysis
   - CLI for command-line usage  
   - Python API for programmatic access
   - Clear function signatures and returns

EXPLAINABILITY METHODS
   - GradientShap: Gradient-based Shapley values
   - Integrated Gradients: Path integration method
   - Saliency: Gradient magnitude highlighting



In [23]:
# Function Return Values
print("=" * 80)
print("FUNCTION RETURN VALUES")
print("=" * 80)

returns = """
load_model(model_path)
└─ Returns: (model, processor, device)

compute_gradient_shap(...)
compute_integrated_gradients(...)
compute_saliency(...)
└─ Returns: dict or None
    {
        'attributions': numpy.ndarray,  # 2D attribution heatmap
        'min': float,                    # Minimum attribution value
        'max': float,                    # Maximum attribution value
        'mean': float,                   # Mean absolute attribution
        'method': str                    # Method name
    }

get_image_files(data_path, num_samples=None)
└─ Returns: list[str]  # List of image file paths

create_transform(processor)
└─ Returns: torchvision.transforms.Compose

load_and_preprocess_image(image_path, transform, device)
└─ Returns: (PIL.Image, torch.Tensor)

get_model_prediction(model, input_tensor)
└─ Returns: (pred_class: int, pred_prob: float)
"""

print(returns)
print("\n✓ All functions have clear, documented return types")
print("✓ Attribution methods return None on error")

FUNCTION RETURN VALUES

load_model(model_path)
└─ Returns: (model, processor, device)

compute_gradient_shap(...)
compute_integrated_gradients(...)
compute_saliency(...)
└─ Returns: dict or None
    {
        'attributions': numpy.ndarray,  # 2D attribution heatmap
        'min': float,                    # Minimum attribution value
        'max': float,                    # Maximum attribution value
        'mean': float,                   # Mean absolute attribution
        'method': str                    # Method name
    }

get_image_files(data_path, num_samples=None)
└─ Returns: list[str]  # List of image file paths

create_transform(processor)
└─ Returns: torchvision.transforms.Compose

load_and_preprocess_image(image_path, transform, device)
└─ Returns: (PIL.Image, torch.Tensor)

get_model_prediction(model, input_tensor)
└─ Returns: (pred_class: int, pred_prob: float)


✓ All functions have clear, documented return types
✓ Attribution methods return None on error


In [24]:

# ================================================================================
# ADVANCED USAGE: ERROR HANDLING & PERFORMANCE
# ================================================================================

advanced_usage = """
ERROR HANDLING PATTERNS
================================================================================

1. Safe Attribution Computation:
   
   result = compute_gradient_shap(model, input_tensor, pred_class, device)
   if result is None:
       print("Attribution failed - check input tensor or model compatibility")
   else:
       attributions = result['attributions']
       print(f"Attribution stats: min={result['min']:.3f}, max={result['max']:.3f}")


2. Robust Batch Processing:
   
   successful = 0
   failed = 0
   
   for img_path in image_files:
       try:
           orig_img, input_t = load_and_preprocess_image(img_path, transform, device)
           pred_c, pred_p = get_model_prediction(model, input_t)
           result = compute_saliency(model, input_t, pred_c, device)
           
           if result:
               successful += 1
           else:
               failed += 1
               print(f"Attribution failed for {img_path}")
       except Exception as e:
           failed += 1
           print(f"Error processing {img_path}: {e}")
   
   print(f"Results: {successful} successful, {failed} failed")


PERFORMANCE OPTIMIZATION
================================================================================

1. Reuse Model (avoids repeated loading):
   
   model, processor, device = load_model(model_path)  # Load once
   transform = create_transform(processor)
   
   for img in images:
       # Reuse model, processor, transform for all images
       ...


2. Limit Samples for Testing:
   
   # Test on small subset first
   image_files = get_image_files('./data', num_samples=5)


3. Choose Faster Methods:
   
   # Saliency is fastest
   explain_model_predictions(model_path, data_path, methods=['saliency'])
   
   # GradientSHAP and Integrated Gradients are slower but more accurate
   explain_model_predictions(model_path, data_path, methods=['gradientshap'])


✓ All compute_* functions return None on error
✓ Always check return values before using attributions
✓ Load model once, reuse for all images
"""

print(advanced_usage)



ERROR HANDLING PATTERNS

1. Safe Attribution Computation:

   result = compute_gradient_shap(model, input_tensor, pred_class, device)
   if result is None:
       print("Attribution failed - check input tensor or model compatibility")
   else:
       attributions = result['attributions']
       print(f"Attribution stats: min={result['min']:.3f}, max={result['max']:.3f}")


2. Robust Batch Processing:

   successful = 0
   failed = 0

   for img_path in image_files:
       try:
           orig_img, input_t = load_and_preprocess_image(img_path, transform, device)
           pred_c, pred_p = get_model_prediction(model, input_t)
           result = compute_saliency(model, input_t, pred_c, device)

           if result:
               successful += 1
           else:
               failed += 1
               print(f"Attribution failed for {img_path}")
       except Exception as e:
           failed += 1
           print(f"Error processing {img_path}: {e}")

   print(f"Results: {successful}

## Method 4: Batch Processing with Custom Logic

Process multiple images with your own logic:

In [25]:

# ================================================================================
# TROUBLESHOOTING & BEST PRACTICES
# ================================================================================

troubleshooting = """
COMMON ISSUES & SOLUTIONS
================================================================================

Issue: "No images found in directory"
Solution: 
  • Check path exists and contains .jpg, .jpeg, .png files
  • Use absolute paths or verify working directory
  • Test with: get_image_files('./path/to/images')


Issue: Attribution computation returns None
Solution:
  • Verify model and input tensor are on same device
  • Check input tensor shape matches model requirements
  • Ensure pred_class is valid integer index


Issue: Memory errors with large batches
Solution:
  • Reduce num_samples or process in smaller batches
  • Use saliency method (faster, less memory)
  • Close matplotlib figures: plt.close('all')


BEST PRACTICES
================================================================================

✓ Always load model once and reuse
✓ Check return values (compute_* functions return None on error)
✓ Use absolute paths for reliability
✓ Start with small num_samples for testing
✓ Choose methods based on speed/accuracy tradeoff:
  - Saliency: Fastest, less accurate
  - Integrated Gradients: Balanced
  - GradientSHAP: Most accurate, slowest

✓ For production: Add try/except blocks around image processing
✓ For research: Use all methods and compare results


QUICK VALIDATION
================================================================================

# Test setup without full analysis:
from src.image import get_image_files, load_model

# Verify images found
images = get_image_files('./data', num_samples=1)
print(f"Found {len(images)} images")

# Verify model loads
model, processor, device = load_model('microsoft/resnet-50')
print(f"Model loaded on {device}")
"""

print(troubleshooting)



COMMON ISSUES & SOLUTIONS

Issue: "No images found in directory"
Solution: 
  • Check path exists and contains .jpg, .jpeg, .png files
  • Use absolute paths or verify working directory
  • Test with: get_image_files('./path/to/images')


Issue: Attribution computation returns None
Solution:
  • Verify model and input tensor are on same device
  • Check input tensor shape matches model requirements
  • Ensure pred_class is valid integer index


Issue: Memory errors with large batches
Solution:
  • Reduce num_samples or process in smaller batches
  • Use saliency method (faster, less memory)
  • Close matplotlib figures: plt.close('all')


BEST PRACTICES

✓ Always load model once and reuse
✓ Check return values (compute_* functions return None on error)
✓ Use absolute paths for reliability
✓ Start with small num_samples for testing
✓ Choose methods based on speed/accuracy tradeoff:
  - Saliency: Fastest, less accurate
  - Integrated Gradients: Balanced
  - GradientSHAP: Most accurate, 

## Summary

This notebook demonstrated:
1. **High-level usage** - `explain_model_predictions()` for complete automated analysis
2. **Method selection** - Choose specific explainability methods
3. **Modular approach** - Use individual functions for custom workflows
4. **Integrated Gradients** - Direct use of `compute_integrated_gradients()`
5. **Method comparison** - Side-by-side comparison with `save_comparison_visualization()`

### All 6 Functions from images.py Demonstrated:
✅ `compute_gradient_shap()` - Example 3 & 5
✅ `compute_integrated_gradients()` - Example 4 & 5  
✅ `compute_saliency()` - Example 3 & 5
✅ `save_attribution_visualization()` - Examples 3 & 4
✅ `save_comparison_visualization()` - Example 5
✅ `explain_model_predictions()` - Examples 1 & 2

### Key Takeaways:
- Use `explain_model_predictions()` for quick, complete analysis
- Use individual `compute_*` functions for custom workflows
- Load model once with `load_model()`, reuse for all images
- Handle both single images and directories with `get_image_files()`
- Compare methods with `save_comparison_visualization()` to understand differences

## Available Functions Reference

### Model & Data Utilities (helpers/image_utils.py)
- `ModelWrapper` - Wrapper class for HuggingFace models
- `load_model(model_path)` - Load and prepare image classification model
- `get_image_files(data_path, num_samples)` - Get image files from directory or single file
- `create_transform(processor)` - Create preprocessing transformation pipeline
- `load_and_preprocess_image(image_path, transform, device)` - Load and preprocess single image
- `get_model_prediction(model, input_tensor)` - Get prediction class and confidence

### Explainability Functions (images.py)
1. **`compute_gradient_shap()`** - Compute GradientShap attributions (gradient-based Shapley values)
2. **`compute_integrated_gradients()`** - Compute Integrated Gradients attributions (path integration)
3. **`compute_saliency()`** - Compute Saliency attributions (gradient magnitude)
4. **`save_attribution_visualization()`** - Save single attribution heatmap overlay
5. **`save_comparison_visualization()`** - Save side-by-side comparison of all methods
6. **`explain_model_predictions()`** - High-level orchestrator for complete analysis

### Typical Workflow
1. Load model → `load_model('microsoft/resnet-50')`
2. Get images → `get_image_files('./data/', num_samples=10)`
3. Create transform → `create_transform(processor)`
4. For each image:
   - Load & preprocess → `load_and_preprocess_image()`
   - Get prediction → `get_model_prediction()`
   - Compute attributions → `compute_gradient_shap()` / `compute_integrated_gradients()` / `compute_saliency()`
   - Visualize → `save_attribution_visualization()` or `save_comparison_visualization()`

Or use `explain_model_predictions()` to do all steps automatically.

### Method Selection Guide
- **GradientShap**: Most accurate, considers baselines, slower (~2-5s per image)
- **Integrated Gradients**: Balanced accuracy/speed, smooth attributions (~1-3s per image)
- **Saliency**: Fastest, simple gradient-based, may be noisy (~0.5s per image)

Use all three for research; use Saliency for quick testing; use GradientShap for production.

## Key Concepts and Interpretations

### Understanding Attribution Maps
- **Hot colors (red/yellow)**: Regions that increase predicted class probability
- **Cool colors (blue/purple)**: Regions that decrease predicted class probability  
- **Intensity**: Strength of the pixel's contribution to the prediction
- **White/transparent**: Pixels with minimal impact on prediction

### Reading Visualizations
- **Heatmap overlay**: Attribution values superimposed on original image
- **Bright regions**: Most important for model's decision
- **Attribution range**: Normalized to [0, 1] for visualization
- **Mean absolute attribution**: Average importance across all pixels

### Explainability Method Differences

#### GradientShap
- **Approach**: Combines gradients with Shapley value sampling
- **Baseline**: Uses multiple baseline images (noise) for comparison
- **Best for**: Identifying precise regions that distinguish the predicted class
- **Interpretation**: "This pixel makes the prediction more/less like class X compared to random noise"

#### Integrated Gradients
- **Approach**: Integrates gradients along path from baseline to input
- **Baseline**: Uses single baseline (typically black image or blurred version)
- **Best for**: Smooth, coherent attribution maps with theoretical guarantees
- **Interpretation**: "This pixel's contribution accumulated from baseline to actual image"

#### Saliency
- **Approach**: Simple gradient magnitude at input
- **Baseline**: No baseline needed (single forward-backward pass)
- **Best for**: Quick identification of high-gradient regions
- **Interpretation**: "This pixel has high sensitivity to output changes"

### Image Classification Specifics
- **Edge detection**: Models often focus on edges and boundaries
- **Texture patterns**: Repeated patterns may show consistent attributions
- **Object parts**: Discriminative parts (eyes, wheels, etc.) typically highlighted
- **Background**: Low attributions indicate irrelevant regions
- **Context**: Sometimes surprising regions matter (e.g., typical backgrounds for certain objects)

### Model Confidence vs. Attribution
- **High confidence + focused attribution**: Model clearly identified key features
- **High confidence + diffuse attribution**: Model uses many subtle cues
- **Low confidence + scattered attribution**: Model uncertain, considering multiple possibilities
- **Low confidence + focused attribution**: Model sees contradictory evidence

## Next Steps

1. **Test Different Models**: Compare ResNet, ViT, EfficientNet explanations
2. **Analyze Failure Cases**: Examine misclassifications with low confidence
3. **Compare Methods**: Use comparison visualizations to understand method differences
4. **Domain-Specific Analysis**: Apply to medical images, materials science, etc.
5. **Batch Processing**: Process large datasets with custom filtering logic
6. **Attribution Statistics**: Analyze attribution patterns across image categories
7. **Model Debugging**: Use attributions to identify model biases or spurious correlations
8. **Fine-tuning Validation**: Verify fine-tuned models focus on correct features

### Advanced Applications
- **Model Comparison**: Compare attributions from different architectures on same images
- **Adversarial Analysis**: Study how attributions change under adversarial perturbations
- **Feature Validation**: Confirm models use domain-relevant features (e.g., medical diagnostics)
- **Trust Building**: Show stakeholders what the model "sees" for transparency

In [32]:
# Example 4: Using compute_integrated_gradients() directly
import sys
import os
sys.path.insert(0, './src')

from helpers.image_utils import (
    load_model,
    get_image_files,
    create_transform,
    load_and_preprocess_image,
    get_model_prediction
)
from images import (
    compute_integrated_gradients,
    save_attribution_visualization
)

print("=" * 80)
print("EXAMPLE 4: Integrated Gradients Method")
print("=" * 80)

# Find an image
possible_paths = ['./data/fracture_samples', './data/generic_samples', './data/sample_images', './images/', './data/']
image_files = []
for path in possible_paths:
    if os.path.exists(path):
        image_files = get_image_files(path, num_samples=1)
        if image_files:
            print(f"\n✓ Found image in: {path}")
            break

if image_files:
    # Load model
    print("\n[Step 1] Loading model...")
    model, processor, device = load_model('microsoft/resnet-50')
    print(f"✓ Model loaded on {device}")
    
    # Process image
    img_path = image_files[0]
    img_name = os.path.basename(img_path)
    print(f"\n[Step 2] Processing: {img_name}")
    
    transform = create_transform(processor)
    original_img, input_tensor = load_and_preprocess_image(img_path, transform, device)
    pred_class, pred_prob = get_model_prediction(model, input_tensor)
    
    print(f"✓ Prediction: Class {pred_class} (confidence: {pred_prob:.2%})")
    
    # Compute Integrated Gradients
    print("\n[Step 3] Computing Integrated Gradients attributions...")
    result = compute_integrated_gradients(
        model, 
        input_tensor, 
        pred_class, 
        device, 
        n_steps=100  # More steps = smoother attributions
    )
    
    if result:
        print(f"✓ Attribution range: [{result['min']:.4f}, {result['max']:.4f}]")
        print(f"✓ Mean absolute attribution: {result['mean']:.4f}")
        
        # Save visualization
        os.makedirs('./outputs/integrated_gradients', exist_ok=True)
        output_path = f"./outputs/integrated_gradients/{os.path.splitext(img_name)[0]}_ig.png"
        
        save_attribution_visualization(
            original_img,
            result['attributions'],
            'Integrated Gradients',
            output_path,
            pred_class,
            pred_prob
        )
        
        print(f"\n✓ Saved: {output_path}")
        print("\nIntegrated Gradients provides smooth, theoretically-grounded attributions")
        print("by integrating gradients along the path from baseline to input.")
    else:
        print("✗ Attribution computation failed")
else:
    print("\nℹ No images found. Add images to run this example.")
    print("\n📝 CODE STRUCTURE:")
    print("""
# Load model once
model, processor, device = load_model('microsoft/resnet-50')
transform = create_transform(processor)

# For each image
original_img, input_tensor = load_and_preprocess_image(img_path, transform, device)
pred_class, pred_prob = get_model_prediction(model, input_tensor)

# Compute Integrated Gradients
result = compute_integrated_gradients(model, input_tensor, pred_class, device, n_steps=100)

# Save visualization
save_attribution_visualization(original_img, result['attributions'], 
                              'Integrated Gradients', output_path, 
                              pred_class, pred_prob)
    """)

EXAMPLE 4: Integrated Gradients Method
Directory detected: ./data/fracture_samples

✓ Found image in: ./data/fracture_samples

[Step 1] Loading model...
Using device: cpu
Loading model: microsoft/resnet-50
✓ Model loaded on cpu

[Step 2] Processing: brick_texture.png
✓ Prediction: Class 828 (confidence: 48.66%)

[Step 3] Computing Integrated Gradients attributions...
Computing Integrated Gradients attributions...
IntegratedGradients attribution range: [0.0001, 0.2681]
IntegratedGradients mean absolute attribution: 0.0154
✓ Attribution range: [0.0001, 0.2681]
✓ Mean absolute attribution: 0.0154
    Saved: ./outputs/integrated_gradients/brick_texture_ig.png

✓ Saved: ./outputs/integrated_gradients/brick_texture_ig.png

Integrated Gradients provides smooth, theoretically-grounded attributions
by integrating gradients along the path from baseline to input.


## Method 5: Demonstrate save_comparison_visualization()

Compare all three methods side-by-side:

In [33]:
# Example 5: Using save_comparison_visualization() to compare all methods
import sys
import os
sys.path.insert(0, './src')

from helpers.image_utils import (
    load_model,
    get_image_files,
    create_transform,
    load_and_preprocess_image,
    get_model_prediction
)
from images import (
    compute_gradient_shap,
    compute_integrated_gradients,
    compute_saliency,
    save_comparison_visualization
)

print("=" * 80)
print("EXAMPLE 5: Method Comparison Visualization")
print("=" * 80)

# Find an image
possible_paths = ['./data/fracture_samples', './data/generic_samples', './data/sample_images', './images/', './data/']
image_files = []
for path in possible_paths:
    if os.path.exists(path):
        image_files = get_image_files(path, num_samples=1)
        if image_files:
            print(f"\n✓ Found image in: {path}")
            break

if image_files:
    # Load model
    print("\n[Step 1] Loading model...")
    model, processor, device = load_model('microsoft/resnet-50')
    print(f"✓ Model loaded on {device}")
    
    # Process image
    img_path = image_files[0]
    img_name = os.path.basename(img_path)
    print(f"\n[Step 2] Processing: {img_name}")
    
    transform = create_transform(processor)
    original_img, input_tensor = load_and_preprocess_image(img_path, transform, device)
    pred_class, pred_prob = get_model_prediction(model, input_tensor)
    
    print(f"✓ Prediction: Class {pred_class} (confidence: {pred_prob:.2%})")
    
    # Compute all three attribution methods
    print("\n[Step 3] Computing all three attribution methods...")
    attributions_dict = {}
    
    print("  • Computing GradientShap...")
    result_gs = compute_gradient_shap(model, input_tensor, pred_class, device, n_samples=50)
    if result_gs:
        attributions_dict['GradientShap'] = result_gs['attributions']
        print(f"    ✓ Range: [{result_gs['min']:.4f}, {result_gs['max']:.4f}]")
    
    print("  • Computing Integrated Gradients...")
    result_ig = compute_integrated_gradients(model, input_tensor, pred_class, device, n_steps=50)
    if result_ig:
        attributions_dict['IntegratedGradients'] = result_ig['attributions']
        print(f"    ✓ Range: [{result_ig['min']:.4f}, {result_ig['max']:.4f}]")
    
    print("  • Computing Saliency...")
    result_sal = compute_saliency(model, input_tensor, pred_class, device)
    if result_sal:
        attributions_dict['Saliency'] = result_sal['attributions']
        print(f"    ✓ Range: [{result_sal['min']:.4f}, {result_sal['max']:.4f}]")
    
    # Create comparison visualization
    if len(attributions_dict) == 3:
        print("\n[Step 4] Creating comparison visualization...")
        os.makedirs('./outputs/comparisons', exist_ok=True)
        output_path = f"./outputs/comparisons/{os.path.splitext(img_name)[0]}_comparison.png"
        
        save_comparison_visualization(
            original_img,
            attributions_dict,
            output_path,
            pred_class,
            pred_prob,
            img_name
        )
        
        print(f"✓ Saved comparison: {output_path}")
        print("\n" + "=" * 80)
        print("COMPARISON INSIGHTS:")
        print("=" * 80)
        print("• GradientShap: Most stable, uses baseline comparisons")
        print("• Integrated Gradients: Smoothest, theoretically grounded")
        print("• Saliency: Fastest, highlights high-gradient regions")
        print("\nView the comparison image to see how each method highlights")
        print("different aspects of the image for the same prediction!")
    else:
        print(f"\n✗ Only {len(attributions_dict)}/3 methods succeeded")
else:
    print("\nℹ No images found. Add images to run this example.")
    print("\n📝 CODE STRUCTURE:")
    print("""
# Compute all three methods
attributions_dict = {}
attributions_dict['GradientShap'] = compute_gradient_shap(...)['attributions']
attributions_dict['IntegratedGradients'] = compute_integrated_gradients(...)['attributions']
attributions_dict['Saliency'] = compute_saliency(...)['attributions']

# Create side-by-side comparison
save_comparison_visualization(
    original_img,
    attributions_dict,
    output_path,
    pred_class,
    pred_prob,
    image_name
)
    """)

EXAMPLE 5: Method Comparison Visualization
Directory detected: ./data/fracture_samples

✓ Found image in: ./data/fracture_samples

[Step 1] Loading model...
Using device: cpu
Loading model: microsoft/resnet-50
✓ Model loaded on cpu

[Step 2] Processing: brick_texture.png
✓ Prediction: Class 828 (confidence: 48.66%)

[Step 3] Computing all three attribution methods...
  • Computing GradientShap...
Computing GradientShap attributions...
GradientShap attribution range: [0.0001, 0.0927]
GradientShap mean absolute attribution: 0.0058
    ✓ Range: [0.0001, 0.0927]
  • Computing Integrated Gradients...
Computing Integrated Gradients attributions...
IntegratedGradients attribution range: [0.0001, 0.2719]
IntegratedGradients mean absolute attribution: 0.0155
    ✓ Range: [0.0001, 0.2719]
  • Computing Saliency...
Computing Saliency attributions...
Saliency attribution range: [0.0007, 1.1264]
Saliency mean absolute attribution: 0.0588
    ✓ Range: [0.0007, 1.1264]

[Step 4] Creating comparison v